In [1]:
import cohere
import sys,os
import pandas as pd
import requests
import datetime
from tqdm import tqdm

In [29]:
df=pd.read_json('../data/relations_dev.json')
db=pd.read_json('../data/relations_test.json')
df.head()

,document,tokens,relations
0,Bachelor's degree in Mechanical Engineering or...,"[{'text': 'Bachelor', 'start': 0, 'end': 8, 't...","[{'child': 4, 'head': 0, 'relationLabel': 'DEG..."
1,10+ years of software engineering work experie...,"[{'text': '10+ years', 'start': 0, 'end': 9, '...","[{'child': 4, 'head': 0, 'relationLabel': 'EXP..."
2,3+ years Swift & Objective-C and experience wi...,"[{'text': '3+ years', 'start': 0, 'end': 8, 't...","[{'child': 3, 'head': 0, 'relationLabel': 'EXP..."
3,8+ years experience in software engineering le...,"[{'text': '8+ years', 'start': 0, 'end': 8, 't...","[{'child': 5, 'head': 0, 'relationLabel': 'EXP..."
4,BS degree in Computer Science or related field...,"[{'text': 'BS', 'start': 0, 'end': 2, 'token_s...","[{'child': 3, 'head': 0, 'relationLabel': 'DEG..."


In [7]:
def preprocess_tokens(self,tokens):
        enti = {}
        for t in tokens:

            if t['entityLabel'] in enti.keys():
                enti[t['entityLabel']] = enti[t['entityLabel']] + ','+t['text']
            else:
                enti[t['entityLabel']] = t['text']
        label = ""
        for k, v in enti.items():
            label += k+':'+v+"\n"
        return label

In [31]:
db['labels']= db['tokens'].apply(preprocess_tokens)
df['labels']= df['tokens'].apply(preprocess_tokens)
df.head()

,document,tokens,relations,labels
0,Bachelor's degree in Mechanical Engineering or...,"[{'text': 'Bachelor', 'start': 0, 'end': 8, 't...","[{'child': 4, 'head': 0, 'relationLabel': 'DEG...",DIPLOMA:Bachelor\nDIPLOMA_MAJOR:Mechanical Eng...
1,10+ years of software engineering work experie...,"[{'text': '10+ years', 'start': 0, 'end': 9, '...","[{'child': 4, 'head': 0, 'relationLabel': 'EXP...","EXPERIENCE:10+ years,5+ years\nSKILLS:software..."
2,3+ years Swift & Objective-C and experience wi...,"[{'text': '3+ years', 'start': 0, 'end': 8, 't...","[{'child': 3, 'head': 0, 'relationLabel': 'EXP...",EXPERIENCE:3+ years\nSKILLS:Swift & Objective-C\n
3,8+ years experience in software engineering le...,"[{'text': '8+ years', 'start': 0, 'end': 8, 't...","[{'child': 5, 'head': 0, 'relationLabel': 'EXP...","EXPERIENCE:8+ years,5+ years\nSKILLS:software ..."
4,BS degree in Computer Science or related field...,"[{'text': 'BS', 'start': 0, 'end': 2, 'token_s...","[{'child': 3, 'head': 0, 'relationLabel': 'DEG...",DIPLOMA:BS\nDIPLOMA_MAJOR:Computer Science\nEX...


In [17]:
train_doc = []
for i in range(df.shape[0]):
    ent = df.labels.iloc[i]
    docu = df.document.iloc[i].replace("\n"," ")
    if len(ent)!=0:
        train_doc.append(docu+"\n\nExtracted Text:" +
                        '\n'+ent+"---\n")

In [19]:
with open('../data/output_entity.txt', 'w') as f:
    for item in train_doc:
        # write each item on a new line
        f.write("%s\n" % item.strip())

In [32]:
def entity_extraction(db:pd.DataFrame):
        test_r = db.document.apply(
            lambda x: x.replace("\n", " ")+'\n\nExtracted Text:').to_list()
        return test_r
test = entity_extraction(db)

In [21]:
# Paste your API key here. Remember to not share publicly
api_key = '9Y0OqZQHpfb7vVrypsRXChE8VZzVAmZMCtmbIrhu'

# Create and retrieve a Cohere API key from os.cohere.ai
co = cohere.Client(api_key)

In [ ]:
class cohereExtractor():
    def __init__(self, examples):
        self.examples = examples

    def make_prompt(self, example):
        examples = self.examples + [example]
        return ("".join([str(exam) for exam in examples]))
    def extract(self, example):
      extraction = co.generate(
          model='large',
          prompt=self.make_prompt(example),
          max_tokens=10,
          temperature=0.1,
          stop_sequences=["\n"])
      return(extraction.generations[0].text[:-1])


coherenewsExtractor = cohereExtractor(train_doc[0:5])


In [33]:

print(coherenewsExtractor.make_prompt(test[-1]))

Bachelor's degree in Mechanical Engineering or Physical Science 3+ years track record of developing or specifying fiber optic cables and connector related products Knowledge of fiber optic component, cabling, and interconnect products, technologies, and standards Experience in statistical data analysis Experience with product life cycle management (PLM) process Experience providing solutions to problems and meeting deadlines Experience engaging stakeholders PREFERRED Advanced degree Experience using a software tool for statistical data analysis such as JMP Experience using Agile as product life-cycle management tool Data center or other mission critical development experience

Extracted Text:
DIPLOMA:Bachelor
DIPLOMA_MAJOR:Mechanical Engineering,Physical Science
EXPERIENCE:3+ years
SKILLS:developing,fiber optic cables,connector related products
---
10+ years of software engineering work experience. Technical experience in release automation engineering, CI/CD or related roles. Experien

In [ ]:
results = []
for text in tqdm(test[-1]):
    try:
        extracted_text = coherenewsExtractor.extract(text)
        print(extracted_text)
        results.append(extracted_text)
    except Exception as e:
        print('ERROR: ', e)

In [35]:
pd.DataFrame(data={'text': test[-1], 'extracted_text': results})

,text,extracted_text
0,"Currently holding a faculty, industry, or gov...",
1,"Currently holding a faculty, industry, or gov...","++, C++11, C++14"
2,"Currently holding a faculty, industry, or gov...",003Cstrongu003E8+ years o
3,"Currently holding a faculty, industry, or gov...",.a.d.i.c.
4,"Currently holding a faculty, industry, or gov...",.a.d.i.c.
...,...,...
542,"Currently holding a faculty, industry, or gov...",akes a holistic approach to the design and dev...
543,"Currently holding a faculty, industry, or gov...",Bachelor's degree in Computer Science or rela...
544,"Currently holding a faculty, industry, or gov...",
545,"Currently holding a faculty, industry, or gov...",andidate must have a Bachelor's degree in Compute
